In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
data_path = '../Data/Max, Samantha, Maria data.xlsx'
xls = pd.ExcelFile(data_path)
df_max = pd.read_excel(xls, sheet_name = 'Max', parse_dates = [3])
df_mar = pd.read_excel(xls, sheet_name = 'Maria', parse_dates = [3])
df_sam = pd.read_excel(xls, sheet_name = 'Samantha', parse_dates = [3])
df_all = [df_max, df_mar, df_sam]

C:\Users\Joseph\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell D2858 is marked as a date but the serial value 6684137.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [4]:
# Max df lacks coupon column; dropping
df_mar.drop(columns = 'coupon', inplace = True)
df_sam.drop(columns = 'Coupon (#)', inplace = True)

In [5]:
# Standarize column names
column_names = ['ID', 'Session', 'ReceiptNum', 'ReceiptDate', 'Item', 'Item2',
                 'Uncertain', 'Unknown', 'Quantity', 'Hit', 'Miss', 'Category', 'Comment']
for df in df_all:
    df.columns = column_names

In [20]:
# Assign data types after cleaning typos
df_sam.loc[df_sam.ReceiptNum == datetime.datetime(1900, 1, 1, 0, 0), 'ReceiptNum'] = 1
df_sam.loc[df_sam['Quantity'] == '??', 'Quantity'] = 1

string_columns = ['Item', 'Item2', 'Category', 'Comment']
for df in df_all:
    df[['ID', 'Session', 'ReceiptNum', 'Quantity']] = df[['ID', 'Session', 'ReceiptNum', 'Quantity']].astype(pd.Int64Dtype())
    df['ReceiptDate'] = pd.to_datetime(df['ReceiptDate'], errors = 'coerce')
    df[string_columns] = df[string_columns].astype(str)
    # clean strings
    for col in string_columns:
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace(r'\(|\)|\?', '')

C:\Users\Joseph\AppData\Local\Temp/ipykernel_7664/4197290296.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(|\)|\?', '')
C:\Users\Joseph\AppData\Local\Temp/ipykernel_7664/4197290296.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(|\)|\?', '')
C:\Users\Joseph\AppData\Local\Temp/ipykernel_7664/4197290296.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'\(|\)|\?', '')


In [7]:
# Fill NaN values
# when receipt number is null, assume all items came from a single basket that session
for df in df_all:
    df['Quantity'].fillna(value = 1, inplace = True)
    df.loc[pd.isna(df['ReceiptNum']), 'ReceiptNum'] = 1

In [8]:
# Drop rows with essential NaN values
# Drop 'Unknown' rows
# Drop rows where 'Comments' contains "duplicate" or "repeat"
for df in df_all:
    df.dropna(subset = ['ID', 'Session', 'Item'], inplace = True)
    df.drop(pd.notnull(df['Unknown']).index)
    df.drop(df['Comment'].str.contains(r'duplicate|repeat').index)